In [1]:
import matplotlib.pyplot as plt
import numpy as np
import quantecon as qe
import random

from numba import njit, int64, float64, prange
from numba.experimental import jitclass
%matplotlib inline

#### Goal

In this program, we are going to replicate Arellano (08) model economy and simulate the process.

#### Model

- Risk averse identical agents/citizens in a country, (assume there is a representitative agent)
- One benevolent government (maximizes representitative agent's expected utility), who has the sole access to foreign assets. 
- Foreign creditors are risk neutral and have deep pocket (i.e. can buy or sell as much asset as they want). Their time discounting is 
- One risk free (exogenously risk free) foreign asset. (**Exogenously incomplete market**)
- The government can default on its debt at the beginning of the period after seeing its endowment shock (**Endogenously incomplete market**)